In [6]:
from docxtpl import * 
from typing import Dict

# Site references

In [7]:
#currency values

from models.econometrics import Cost, Currency

#reference 1 is USD
Cost.set_exchange(Currency.CLP,908.72)
Cost.set_exchange(Currency.EUR,0.92)

cost_inverter = Cost(1000,Currency.EUR)
cost_inverter.cost_after_tax(Currency.CLP)

# exchange rates https://app.freecurrencyapi.com/dashboard


987929.1304347826

In [8]:
#cspell:disable


#init proyect
from models.components import Tech
from models.geometry import GeoPosition
from models.inventory import Building, Project

psr_bucalemu:Project = Project(
    building = Building(
        geolocation=GeoPosition(latitude=-33.861702,longitude=-71.657839),
        name='Posta de Salud Rural Bucalemu',
        address='Ruta G-80-I km 30 sn, Localidad Bucalemu, Santo Domingo',
        city='Santo Domingo'),
    technology= [Tech.PHOTOVOLTAIC]
    )


# psr_sanenrique:Project = Project(
#     building = Building(
#         geolocation=GeoPosition(latitude=-33.9081421,longitude=-71.7159192),
#         name='Posta de Salud Rural de San Enrique',
#         address='Ruta G-866 km 0 sn, Localidad San Enrique, Santo Domingo',
#         city='Santo Domingo'),
#     technology= [Tech.PHOTOVOLTAIC]
#     )



api request URL https://power.larc.nasa.gov/api/temporal/hourly/point?Time=LST&parameters=T2M,ALLSKY_SFC_SW_DNI,ALLSKY_SFC_SW_DIFF,ALLSKY_SRF_ALB,SZA,WS10M&community=RE&latitude=-33.861702&longitude=-71.657839&start=20230101&end=20230102&format=JSON


In [9]:
psr_bucalemu.weather.get_data().head(24)

,date,T2M,ALLSKY_SFC_SW_DNI,ALLSKY_SFC_SW_DIFF,ALLSKY_SRF_ALB,SZA,WS10M,year,month,day,hour
0,2022-12-31 19:00:00,15.99,0.00,0.00,None,None,1.61,2022,12,31,19
1,2022-12-31 20:00:00,15.62,0.00,0.00,None,None,1.69,2022,12,31,20
2,2022-12-31 21:00:00,15.29,0.00,0.00,None,None,1.91,2022,12,31,21
3,2022-12-31 22:00:00,14.89,0.00,0.00,None,None,2.08,2022,12,31,22
4,2022-12-31 23:00:00,14.46,0.00,0.00,None,None,2.37,2022,12,31,23
5,2023-01-01 00:00:00,14.20,0.00,0.00,None,None,2.73,2023,01,01,00
6,2023-01-01 01:00:00,13.96,0.00,0.00,None,None,2.96,2023,01,01,01
7,2023-01-01 02:00:00,13.66,0.00,0.00,None,None,2.99,2023,01,01,02
8,2023-01-01 03:00:00,13.34,0.00,0.00,None,None,3.09,2023,01,01,03
9,2023-01-01 04:00:00,13.16,0.00,0.00,None,None,3.15,2023,01,01,04


# Integration components

In [16]:

import math
from models.components import Component
from models.econometrics import Cost
from models.photovoltaic import Photovoltaic
#cspell: disable

pv_log_model = lambda size_w: math.floor(-152.6*math.log(size_w)+2605.9) # clp/w

# assembly projects components
psr_bucalemu.add_component(
    'generación',
    Photovoltaic(psr_bucalemu.weather,'Módulo A Principal',power=250,quantity=40,cost_model=pv_log_model),
    Photovoltaic(psr_bucalemu.weather,'Módulo Secundario B',power=250,quantity=20,cost_model=pv_log_model)
    )

psr_bucalemu.add_component(
    'auxiliar',
    Component('inversor de 5kW'),
    Component('regulador'),
    )
psr_bucalemu.add_component(
    'mano de obra',
    Component('instalación','eléctrica',None,Cost(100000)),
    Component('obras menores','habilitación de espacios',None,Cost(100000))
)

NameError: name 'Math' is not defined

In [15]:
psr_bucalemu.components
pv_log_model(2000) #clp/w

1446.0022846738784